Imports

In [1]:
from configParser import ConfigParser, getModelName, getDatasetName

Parameters

In [2]:
experimentName = "test_22987_suffix50_11_CNN2_differentConfig_baseline4fclayers"
numOfExperiments=100

params = {
    # Do not change for multi/hyperp experiments
    "image_path": "/data/BGNN_data/INHS_cropped",
    "suffix": '50_11', # used to get a subset cleaned_metadata file. set to None otherwise to use full metadata
    
    # dataset
    "training_count":0.64,
    "validation_count":0.16,
    
    # image
    "imageDimension": 224, # Deprecated and should not be changed from 224
    "n_channels": 3,
    "augmentation": False, # False is well better for Hierarchy

    # training
    "batchSize":550, # Bigger is more stable
    "n_epochs":4000, # allow it to run for long time 4000
    "learning_rate":0.01, # Seems 0.0001 to be best value with Adam. 0.1 best for SGD
    "numOfTrials":3,
    "patience": 100,
    "useHeirarchy": False, #True: This makes a big good difference!
    # temp. Should be optimized and removed
    "useRelu": True, # True slows it down with no benefit,
    "downsample":True, #False,
    "downsampleOutput":200, #200 looks best marginally
    "takeFromIntermediate":True, #True,
    "takeFromIntermediateOutput":200, #200 looks best marginally
    "fc_layers": 4, # 2 seems to be best. definitely better than 1
    "useAdam": False, # Keep it to True for now!
    "resnet": "18", # Keep '18'. '50' is not bad but not as good.
    "normalize": True, # Does not seem to matter with no augmentation. Keep False for now
    "softmax": True, # keep True for interpretability
    "batchNormalize": True, # Keep True
}


Create file

In [3]:
cp = ConfigParser(experimentName)

cp.write(params)

'test_22987_suffix50_11_CNN2_differentConfig_baseline4fclayers/params.json'

Read file

In [4]:
j = cp.read()
print(j)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': False, 'useRelu': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'fc_layers': 4, 'useAdam': False, 'resnet': '18', 'normalize': True, 'softmax': True, 'batchNormalize': True}


Create hyperp

In [5]:
hyperp_params = {
#     "dummy":[0]
#     "fc_layers":[1,2,3]
#     "imageDimension": [28, 56, 224],
#     "n_channels": [1, 3],
#     "kernelSize": [5, 9],
#     "useZCAWhitening": [False, True],
#     "patience": [50, 100],
#     "batchSize": [50, 100, 200, 550],
#     "useHeirarchy": [True, False],
#     "resnet": ['18', '50']
    "batchNormalize": [False, True]
}

cp.writeHyperp(hyperp_params)

'test_22987_suffix50_11_CNN2_differentConfig_baseline4fclayers/hyperp_params.json'

Get experiments

In [6]:
for i in cp.getHyperpIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': False, 'useRelu': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'fc_layers': 4, 'useAdam': False, 'resnet': '18', 'normalize': True, 'softmax': True, 'batchNormalize': False}
{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': False, 'useRelu': True, 'downsample': True, 'downsampleOutput': 200, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'fc_layers': 4, 'useAdam': False, 'resnet':

Model name

In [7]:
print(getModelName(j))
print(getDatasetName(j))

tc0.640000_vc0.160000_d224_c3_augFalse_nTrue_bs550_e4000_p100_lr0.010000_hFalse_reluTrue_dsTrue_dso200_tfiTrue_tfio200_admFalse_res18_fc4_smaxTrue_bnormTrue
tc0.640000_vc0.160000_d224_c3_augFalse_nTrue


Hyperp search write

In [8]:
from hyperopt import fmin, hp, rand, tpe
from hyperopt.pyll.base import scope
import math

useRandomSearch = True
hyperp_search_params = {
#     "kernels": [ scope.int(hp.quniform('kernels1', 8, 16, 1)),
#                 scope.int(hp.quniform('kernels2', 16, 32, 1)),
#                 scope.int(hp.qloguniform('kernels3', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels4', math.log(1), math.log(16), 1)),
#                 scope.int(hp.qloguniform('kernels5', math.log(1), math.log(16), 1))],
#     "kernelSize": scope.int(hp.quniform('kernelSize', 3, 10, 1)), 
    # "useZCAWhitening": hp.choice('useZCAWhitening', [False, True]), 
#     "n_channels": scope.int(hp.choice('n_channels', [1, 3])), 
#     "imageDimension": scope.int(hp.quniform('imageDimension', 28, 420, 28)),
    "batchSize": scope.int(hp.quniform('batchSize', 50, 550, 50)),
    "learning_rate": hp.loguniform('learning_rate', math.log(0.0001), math.log(0.01)),
    "patience": scope.int(hp.quniform('patience', 50, 500, 25)),
}

cp.writeHyperpSearch(hyperp_search_params)

'test_22987_suffix50_11_CNN2_differentConfig_baseline4fclayers/hyperp_search_params.pkl'

Hyperp search read

In [9]:
hyperp_obj = cp.getHyperpSearchObject()
print(hyperp_obj)

def objective(experiment_params):
    print(experiment_params)
    dummyObjective = 1
    return dummyObjective

fmin(objective,
     space=hyperp_obj,
     algo=rand.suggest, 
     max_evals=numOfExperiments)

{'batchSize': <hyperopt.pyll.base.Apply object at 0x7fe3df9a7be0>, 'learning_rate': <hyperopt.pyll.base.Apply object at 0x7fe3df9a7da0>, 'patience': <hyperopt.pyll.base.Apply object at 0x7fe3df92f438>, 'image_path': <hyperopt.pyll.base.Apply object at 0x7fe3df9a7b00>, 'suffix': <hyperopt.pyll.base.Apply object at 0x7fe3df92f710>, 'training_count': <hyperopt.pyll.base.Apply object at 0x7fe3df92f860>, 'validation_count': <hyperopt.pyll.base.Apply object at 0x7fe3df92f9b0>, 'imageDimension': <hyperopt.pyll.base.Apply object at 0x7fe3df92fb00>, 'n_channels': <hyperopt.pyll.base.Apply object at 0x7fe3df92fc50>, 'augmentation': <hyperopt.pyll.base.Apply object at 0x7fe3df92fda0>, 'n_epochs': <hyperopt.pyll.base.Apply object at 0x7fe3df92fef0>, 'numOfTrials': <hyperopt.pyll.base.Apply object at 0x7fe3df932080>, 'useHeirarchy': <hyperopt.pyll.base.Apply object at 0x7fe3df9321d0>, 'useRelu': <hyperopt.pyll.base.Apply object at 0x7fe3df932320>, 'downsample': <hyperopt.pyll.base.Apply object at 0

{'augmentation': False, 'batchNormalize': True, 'batchSize': 100, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0015908360577506072, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 125, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 550, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0008769784324126549, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 425, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'trainin

{'augmentation': False, 'batchNormalize': True, 'batchSize': 450, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00018168861828851799, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 425, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 300, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00015947513881585406, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 450, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'train

{'augmentation': False, 'batchNormalize': True, 'batchSize': 200, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0005672931956968139, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 250, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 150, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0002640399118134493, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 500, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'trainin

{'augmentation': False, 'batchNormalize': True, 'batchSize': 100, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00023125009754471773, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 300, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 300, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.00093840571098105, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 100, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training

{'augmentation': False, 'batchNormalize': True, 'batchSize': 100, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0013810029429236115, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 300, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 200, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.007574398003833564, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 300, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training

{'augmentation': False, 'batchNormalize': True, 'batchSize': 250, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0025135527772671184, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 350, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'training_count': 0.64, 'useAdam': False, 'useHeirarchy': False, 'useRelu': True, 'validation_count': 0.16}
{'augmentation': False, 'batchNormalize': True, 'batchSize': 350, 'downsample': True, 'downsampleOutput': 200, 'fc_layers': 4, 'imageDimension': 224, 'image_path': '/data/BGNN_data/INHS_cropped', 'learning_rate': 0.0004889909723795954, 'n_channels': 3, 'n_epochs': 4000, 'normalize': True, 'numOfTrials': 3, 'patience': 400, 'resnet': '18', 'softmax': True, 'suffix': '50_11', 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 200, 'trainin

{'augmentation': 0,
 'batchNormalize': 0,
 'batchSize': 300.0,
 'downsample': 0,
 'downsampleOutput': 0,
 'fc_layers': 0,
 'imageDimension': 0,
 'image_path': 0,
 'learning_rate': 0.0030567278364565017,
 'n_channels': 0,
 'n_epochs': 0,
 'normalize': 0,
 'numOfTrials': 0,
 'patience': 325.0,
 'resnet': 0,
 'softmax': 0,
 'suffix': 0,
 'takeFromIntermediate': 0,
 'takeFromIntermediateOutput': 0,
 'training_count': 0,
 'useAdam': 0,
 'useHeirarchy': 0,
 'useRelu': 0,
 'validation_count': 0}

Selected hyperp write

In [10]:
selected_hyperp_params = [
    {"downsampleOutput":50, "takeFromIntermediateOutput":50,},
    {"downsampleOutput":100, "takeFromIntermediateOutput":100,},
    {"downsampleOutput":200, "takeFromIntermediateOutput":200,},
    {"downsampleOutput":300, "takeFromIntermediateOutput":300,},
]

cp.writeHyperpSelected(selected_hyperp_params)

'test_22987_suffix50_11_CNN2_differentConfig_baseline4fclayers/hyperp_selected_params.pkl'

Selected hyperp read

In [11]:
for i in cp.getHyperpSelectedIter():
    print(i)

{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': False, 'useRelu': True, 'downsample': True, 'downsampleOutput': 50, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 50, 'fc_layers': 4, 'useAdam': False, 'resnet': '18', 'normalize': True, 'softmax': True, 'batchNormalize': True}
{'image_path': '/data/BGNN_data/INHS_cropped', 'suffix': '50_11', 'training_count': 0.64, 'validation_count': 0.16, 'imageDimension': 224, 'n_channels': 3, 'augmentation': False, 'batchSize': 550, 'n_epochs': 4000, 'learning_rate': 0.01, 'numOfTrials': 3, 'patience': 100, 'useHeirarchy': False, 'useRelu': True, 'downsample': True, 'downsampleOutput': 100, 'takeFromIntermediate': True, 'takeFromIntermediateOutput': 100, 'fc_layers': 4, 'useAdam': False, 'resnet': '1